# Compute OPR, Ranking point, and Win ratings

In [1]:
from __future__ import print_function
import pickle
#import time
import sys
sys.path.append('..')
import swagger_client as v3client
from swagger_client.rest import ApiException
from collections import Counter
from IPython.display import HTML

In [2]:


HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

Usage:
1. ensure match data is up to date by manually running fetchMatches.py.
2. `jupyter notebook` to launch the notebook browser.
3. Browse to this notebook. Click the button above to toggle on/off code view (off is easier to navigate).
4. Edit the events list in the cell below to limit analysis to the events you care about. Leave events empty to process all 2020 data.

In [3]:

year = 2022
events = []
#events = ['2020waahs', '2020wasno', '2020pncmp']

filename = 'data/matches_{}.pkl'.format(year)
teamname = 'data/teams_{}.pkl'.format(year)
rankname = 'data/rankings_{}.pkl'.format(year)

rankings = {}
with open(rankname, 'rb') as f:
    rankings = pickle.load(f)

matches = []
with open(filename, 'rb') as f:
    matches = pickle.load(f)


teams = []
with open(teamname, 'rb') as f:
    teams = pickle.load(f)
    
teamlookup = dict([(t.key,t.nickname) for t in teams])
    
print('Loaded {} events and {} teams'.format(len(matches['matches']),len(teams)))

FileNotFoundError: [Errno 2] No such file or directory: 'data/rankings_2022.pkl'

In [7]:
import json
with open('matches.json','w', encoding='utf-8') as outF:
    outF.write(json.dumps([m.to_dict() for m in matches['matches']['2020wasno']]))


In [8]:
if (len(events)>0):
    print("Using only events {}".format(','.join(events)))
non_empty = [k for k in matches['matches'].keys() if len(matches['matches'][k])>0]
data = [m for k in matches['matches'] for m in matches['matches'][k] if k in events or len(events)==0]
data = [m for m in data if m.winning_alliance!='']
#print(len(data))
red = [x for m in data for x in m.alliances.red.team_keys]
blue = [x for m in data for x in m.alliances.blue.team_keys]

Counter(red+blue)

# qualifying matches
qualifiers = [m for m in data if m.comp_level=='qm']
alliances = [' '.join(m.alliances.red.team_keys) for m in qualifiers]  + [' '.join(m.alliances.blue.team_keys) for m in qualifiers]
scores = [m.alliances.red.score for m in qualifiers] + [m.alliances.blue.score for m in qualifiers]
opp_scores = [-m.alliances.blue.score for m in qualifiers] + [-m.alliances.red.score for m in qualifiers]
rp = [m.score_breakdown and m.score_breakdown['red']['rp'] or 0.0 for m in qualifiers] + [m.score_breakdown and m.score_breakdown['blue']['rp'] or 0.0 for m in qualifiers]
wins = [int(m.winning_alliance=='red') for m in qualifiers] + [int(m.winning_alliance=='blue') for m in qualifiers]

# finals (we'll ignore these for now)
finals = [m for m in data if m.comp_level!='qm']
alliances_finals = [' '.join(m.alliances.red.team_keys) for m in finals]  + [' '.join(m.alliances.blue.team_keys) for m in finals]
scores_finals = [m.alliances.red.score for m in finals] + [m.alliances.blue.score for m in finals]
#len(scores),len(scores_finals)  # should be 2x number of qualifiers
print('{} total matches'.format(len(data)))

89 total matches


In [9]:
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
import numpy as np
#from scipy.sparse.linalg import spsolve

# two count vectorizers. This transforms the alliance lists into vector encodings
vectorizer = CountVectorizer(max_features=15000, min_df=1, max_df=1.0, stop_words=None)  

X = vectorizer.fit_transform(alliances)
y = np.column_stack([scores, rp, wins, opp_scores])

# we'll ignore for now
X_finals = vectorizer.transform(alliances_finals)
y_finals = scores_finals

#X[1,:].todense()
#np.shape(y)


In [10]:
from scipy.sparse.linalg import lsqr

oprating = lsqr(X,y[:,0])[0]
rprating = lsqr(X,y[:,1])[0]
winrating = lsqr(X,y[:,2])[0]
defrating = lsqr(X,y[:,3])[0]
ratings_all = np.column_stack([oprating, rprating, winrating, defrating])
rating_lookup=dict(zip(sorted(vectorizer.vocabulary_,key=lambda x: vectorizer.vocabulary_[x]),ratings_all))

    
#print(rating_lookup['frc492'])

In [11]:
teamAggregates = {}

def addMatch(team, m):    
    if team not in teamAggregates:       
        # initialize an empty record for the team.
        teamAggregates[team]= {
            'name': teamlookup[team] if team in teamlookup else 'unknown',
            'matches':0, 
            'rp': 0,
            'points': 0,
            'wins': 0
        }
            

    alliance = 'blue' if team in m.alliances.blue.team_keys else 'red'
    points = m.score_breakdown[alliance]
    summary = teamAggregates[team]
    # update all of the fields.
    summary['matches']+=1    
    
    summary['rp'] += points['rp']
    summary['points'] += points['totalPoints']
    summary['wins'] += int(m.winning_alliance==alliance)

# Currently just using Week0 events.
for m in qualifiers:
    if m.score_breakdown is None:
        continue
    # only aggregate statistics for regional matches- skip the districts or we're cheating.
    #if m.event_key=='2020pncmp':
    #    continue
    for t in m.alliances.red.team_keys:
        addMatch(t,m)
    for t in m.alliances.blue.team_keys:
        addMatch(t,m)


ratings = ['op_rating', 'rp_rating', 'win_rating', 'dp_rating']
rating_ranks = {}
for i,r in enumerate(ratings):
    rating_ranks[r] = dict([(t,k+1) for k,t in enumerate(sorted(teamAggregates, key=lambda x: rating_lookup[x][i], reverse=True))])


    
# normalize the aggregates
skipped = 0
for t in teamAggregates:
    for k in teamAggregates[t]:
        if k=='matches' or k=='name':
            continue
        teamAggregates[t][k]/=teamAggregates[t]['matches']    
    if t in rating_lookup:
        for i,r in enumerate(ratings):
            teamAggregates[t][r]=rating_lookup[t][i]
            teamAggregates[t][r.replace('_rating','_rank')] = rating_ranks[r][t]
    else:
        skipped+=1
    if t in rankings['2020wasno']:
        teamAggregates[t]['rank']=rankings['2020wasno'][t]

print('Skipped {} teams with missing data'.format(skipped))

with open('teamStats_tmp_2020.pkl','wb') as f:
    pickle.dump(teamAggregates,f)

Skipped 0 teams with missing data


In [12]:
# Save to tsv for processing in excel.
with open('data/stats_{}.tsv'.format(year),'w',encoding='utf-8') as outStats:    
    header = None
    for t in teamAggregates:
        k = teamAggregates[t]
        #if 'opr' not in k:
        #    continue        
        if header is None:
            header = list(teamAggregates[t].keys())
            outStats.write('team\t{}\n'.format('\t'.join(header)))
        outStats.write('{}\t{}\n'.format(t,'\t'.join([str(k[h]) for h in header])))

In [13]:
from pandas import DataFrame

df = DataFrame.from_dict(dict(filter(lambda x: 'op_rating' in x[1],teamAggregates.items())), orient='index') 
df.index.name = 'team'


In [14]:
"""
Copyright 2019, Marek Cermak
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""

def init_datatable_mode():
    """Initialize DataTable mode for pandas DataFrame represenation."""
    import pandas as pd
    from IPython.core.display import display, Javascript

    # configure path to the datatables library using requireJS
    # that way the library will become globally available
    display(Javascript("""
        require.config({
            paths: {
                DT: '//cdn.datatables.net/1.10.19/js/jquery.dataTables.min',
            }
        });
        $('head').append('<link rel="stylesheet" type="text/css" href="//cdn.datatables.net/1.10.19/css/jquery.dataTables.min.css">');
    """))

    def _repr_datatable_(self):
        """Return DataTable representation of pandas DataFrame."""
        # classes for dataframe table (optional)
        classes = ['table', 'table-striped', 'table-bordered']

        # create table DOM
        script = (
            f'$(element).html(`{self.to_html(index=True, classes=classes, float_format=lambda x: "%.2f" % x)}`);\n'
        )

        # execute jQuery to turn table into DataTable
        script += """
            require(["DT"], function(DT) {
                $(document).ready( () => {
                    // Turn existing table into datatable
                    $(element).find("table.dataframe").DataTable();
                })
            });
        """

        return script

    pd.DataFrame._repr_javascript_ = _repr_datatable_

In [15]:
import numpy as np
import pandas as pd
init_datatable_mode()  # initialize [DataTables]
df

<IPython.core.display.Javascript object>

,name,matches,rp,points,wins,op_rating,op_rank,rp_rating,rp_rank,win_rating,win_rank,dp_rating,dp_rank,rank
team,,,,,,,,,,,,,,
frc1294,Top Gun,12,0.750000,66.750000,0.250000,11.456784,31,0.037582,32,-0.075800,35,-36.042987,30,32
frc1318,Issaquah Robotics Society,12,1.333333,90.583333,0.583333,32.044786,12,0.575807,10,0.248730,11,-13.658066,4,8
frc1778,Chill Out,12,1.250000,91.166667,0.583333,33.932934,10,0.406954,14,0.206583,15,-31.271205,22,12
frc1899,Saints Robotics,12,1.250000,89.333333,0.583333,35.869308,9,0.478725,13,0.217011,14,-36.654662,33,11
frc2412,Robototes,12,1.000000,70.916667,0.500000,13.080205,29,0.286783,19,0.172773,17,-18.362990,9,21
frc2522,Royal Robotics,12,1.833333,98.416667,0.750000,41.527075,6,1.028114,4,0.388551,4,-36.103879,31,4
frc2903,NeoBots,12,0.666667,52.666667,0.333333,1.722148,37,-0.010784,34,0.012452,33,-44.456266,36,34
frc2910,Jack in the Bot,12,2.000000,137.666667,0.916667,84.537824,1,1.132684,3,0.506277,2,-15.428835,7,2
frc2928,Viking Robotics,12,0.166667,54.333333,0.083333,9.408515,34,-0.496512,37,-0.213272,37,-49.183139,37,37
